## 💼 "All-Weather" Risk-Parity Portfolio Framework

### 📥 Setups and ETFs Selection

Import of required packages

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Initialization of portfolio parameters

In [ ]:
TICKERS = ['SPY', 'TLT', 'GLD', 'TIP', 'BIL']  # Equities, Fixed Income, Commodities, Inflation-protected, Cash
START_DATE = '2015-01-01'
END_DATE = '2025-06-27'
VOL_LOOKBACK = 126  # Volatility Lookback Period Length

### 🏗️ Data Download

Function for fetching daily price data

In [56]:
def get_etf_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed adjusted closing price data
    """
    
    data = yf.download(tickers, start, end, auto_adjust=True)['Close']
    return data.dropna()

### 🧮 Volatility and Weight Calculation

Functions for calculating rolling volatility and daily risk-parity weights

In [53]:
def calculate_rolling_volatility(tickers: list[str], start: str, end: str, window: int):
    """
    Calculates rolling 126-day volatility for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed rolling 126-day volatility
    """

    # Fetch price data
    data = get_etf_data(tickers, start, end)

    # Compute daily return
    returns = data.pct_change()

    # Compute rolling volatility
    rolling_vol = returns.rolling(window=window).std()

    return rolling_vol

In [ ]:
def calculate_daily_risk_parity_weights(tickers: list[str] = TICKERS, start: str = START_DATE, end: str = END_DATE, window: int = VOL_LOOKBACK):
    """
    Calculates daily risk-parity weights for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start (str): Start date in 'YYYY-MM-DD' format
        end (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed risk-parity weights
    """

    # Fetch rolling volatility
    vol = calculate_rolling_volatility(tickers, start, end, window)

    # Invert volatility
    inv_vol = 1 / vol.replace(0, np.nan)  # Replace zero vol with NaN to avoid inf

    # Normalize rows so weights sum to 1
    weights = inv_vol.div(inv_vol.sum(axis=1), axis=0).fillna(0)  # Fill NaN weights with zero

    return weights

### 📜 Backtesting

Functions for calculating risk-parity portfolio returns and performance metrics

In [ ]:
def calculate_portfolio_returns(weights: pd.DataFrame, data: pd.DataFrame):
    """
    Calculates daily risk-parity portfolio returns for given weights and total-return ETF prices
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        data (pd.DataFrame): Daily adjusted closing price of each ETF

    Returns:
        pd.DataFrame: DataFrame with date-indexed daily returns
    """

    # Compute daily return
    returns = data.pct_change().dropna()

    # Shift weights to avoid look-ahead bias
    weights_lagged = weights.shift(1).dropna()

    # Align returns with weights
    returns = returns.loc[weights_lagged.index]

    # Calculate portfolio returns
    portfolio_returns = (weights_lagged * returns).sum(axis=1).to_frame()

    portfolio_returns.columns = ['returns']

    return portfolio_returns

In [ ]:
def compute_portfolio_metrics(portfolio_returns: pd.DataFrame, risk_free_rate: float = 0.0) -> dict:
    """
    Computes annualized return, volatility, Sharpe ratio, and max drawdown.

    Parameters:
        portfolio_returns (pd.DataFrame): Must have a column 'returns' of daily returns.
        risk_free_rate (float): Daily risk-free rate in decimal (e.g., 0.0 or 0.0001 for 2.5% annualized)

    Returns:
        dict: Dictionary of metrics
    """

### 📈 Performance Plots

### 🧪 Stress-Testing Under Extreme Scenarios

In [ ]:
# def stress_test_period(portfolio_returns, start, end)

# def compute_var_cvar(returns, alpha=0.05)